In [1]:
import numpy as np
from sklearn.metrics import classification_report
import tensorflow as tf
import tensorflow.keras.layers as layers
from tensorflow.keras.models import Sequential

import cv2

In [2]:
train_ds = tf.keras.utils.image_dataset_from_directory(
  'data/dataset/',
  label_mode='categorical',
  shuffle=True,
  seed=42,
  image_size=(32, 32),
)

test_ds = tf.keras.utils.image_dataset_from_directory(
  'data/test_dataset/',
  label_mode='categorical',
  shuffle=True,
  seed=42,
  image_size=(32, 32),
)

def transform_dataset(x, y):
  # gray = cv2.cvtColor(x, cv2.COLOR_RGB2GRAY)
  print(x.shape)
  print(y)
  return x

# train_ds = train_ds.map(transform_dataset)


Found 10019 files belonging to 10 classes.
Found 353 files belonging to 10 classes.


In [3]:
x, y = next(iter(train_ds))

In [10]:
model = Sequential([
    layers.Input(shape=(None, None, 3)),
    layers.Resizing(32, 32),
    layers.Rescaling(1./255),
    # layers.RandomBrightness(0.2, value_range=(0, 1)),
    layers.RandomFlip('horizontal'),
    # layers.RandomRotation(0.2),
    # layers.RandomZoom(),
    # layers.RandomTranslation(),
    # layers.RandomContrast(),
    # layers.RandomCrop()
    layers.Conv2D(32, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.Flatten(),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(10, activation='softmax'),
])

In [11]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resizing_2 (Resizing)       (None, 32, 32, 3)         0         
                                                                 
 rescaling_2 (Rescaling)     (None, 32, 32, 3)         0         
                                                                 
 random_flip_2 (RandomFlip)  (None, 32, 32, 3)         0         
                                                                 
 conv2d_6 (Conv2D)           (None, 30, 30, 32)        896       
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 15, 15, 32)       0         
 2D)                                                             
                                                                 
 conv2d_7 (Conv2D)           (None, 13, 13, 64)        18496     
                                                      

In [12]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [13]:
history = model.fit(train_ds, validation_data=test_ds, epochs=40, batch_size=64)

Epoch 1/40
314/314 [==============================] - 12s 36ms/step - loss: 1.9184 - accuracy: 0.2720 - val_loss: 2.3713 - val_accuracy: 0.2720
Epoch 2/40
314/314 [==============================] - 12s 37ms/step - loss: 1.1582 - accuracy: 0.5831 - val_loss: 4.2714 - val_accuracy: 0.2833
Epoch 3/40
314/314 [==============================] - 12s 37ms/step - loss: 0.7665 - accuracy: 0.7347 - val_loss: 4.1444 - val_accuracy: 0.3201
Epoch 4/40
314/314 [==============================] - 12s 38ms/step - loss: 0.5193 - accuracy: 0.8235 - val_loss: 4.0571 - val_accuracy: 0.3541
Epoch 5/40
314/314 [==============================] - 12s 38ms/step - loss: 0.3789 - accuracy: 0.8753 - val_loss: 5.3316 - val_accuracy: 0.3229
Epoch 6/40
314/314 [==============================] - 11s 36ms/step - loss: 0.2591 - accuracy: 0.9172 - val_loss: 3.3974 - val_accuracy: 0.3513
Epoch 7/40
314/314 [==============================] - 12s 36ms/step - loss: 0.1836 - accuracy: 0.9426 - val_loss: 5.0097 - val_accuracy:

In [79]:
X, y = zip(*test_ds)
X = np.concatenate(X)
y = np.concatenate(y)

In [90]:
a, b = next(iter(train_ds))

In [203]:
X.shape, y.shape

((72, 32, 32, 3), (72, 10))

In [14]:
X, y = zip(*test_ds)
X = np.concatenate(X)
y = np.concatenate(y)

predictions = model.predict(X)
report = classification_report(np.argmax(y, axis=1), np.argmax(predictions, axis=1))
print(report)

12/12 [==============================] - 0s 9ms/step
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         1
           1       0.00      0.00      0.00         1
           2       0.00      0.00      0.00         1
           3       0.36      0.86      0.51        50
           4       0.83      0.38      0.52        50
           5       0.34      0.44      0.39        50
           6       0.29      0.20      0.24        50
           7       0.62      0.10      0.17        50
           8       0.31      0.10      0.15        50
           9       0.44      0.48      0.46        50

    accuracy                           0.36       353
   macro avg       0.32      0.26      0.24       353
weighted avg       0.45      0.36      0.34       353

